import data

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [26]:
(X_train,y_train),(X_test,y_test) = tf.keras.datasets.mnist.load_data()

In [27]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [28]:
# cast from float64 to float32

X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

# add channels dimension
X_train = X_train[...,None]
X_test = X_test[..., None]

In [29]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(60000, 28, 28, 1)
(60000,)
(10000, 28, 28, 1)
(10000,)


symbolically generate a batch of images and labels

In [30]:
X_tarin_or_test = tf.placeholder(tf.float32, shape=[None,28,28,1], name='input_image')
y_train_or_test = tf.placeholder(tf.int32, shape=[None],name='image_label' )

batch_size = 32

dataset = tf.data.Dataset.from_tensor_slices((X_tarin_or_test,y_train_or_test))

In [31]:
dataset = dataset.batch(batch_size)
batch_generator = dataset.make_initializable_iterator()
X_batch, y_batch = batch_generator.get_next()

In [32]:
y_batch

<tf.Tensor 'IteratorGetNext_1:1' shape=(?,) dtype=int32>

given a symbolic batch, symbolically process it through a network and output the loss

In [34]:
net = X_batch

# add conv layers

layer1 = tf.layers.conv2d(net,filters=32,kernel_size=3,padding='SAME',activation=tf.nn.relu)
layer2 = tf.layers.conv2d(layer1,filters=32,kernel_size=3,padding='SAME',activation=tf.nn.relu)
layer3 = tf.layers.max_pooling2d(layer2,pool_size=2,strides=2)

layer4 = tf.layers.conv2d(layer3,filters=32,kernel_size=3,padding='SAME',activation=tf.nn.relu)
layer5 = tf.layers.conv2d(layer4,filters=32,kernel_size=3,padding='SAME',activation=tf.nn.relu)
layer6 = tf.layers.max_pooling2d(layer5,pool_size=2,strides=2)

# fully connected layer:
fc = tf.layers.flatten(layer6)
fc = tf.layers.dense(fc,units=512,activation=tf.nn.relu)
logits = tf.layers.dense(fc,units=10)

In [35]:
loss = tf.losses.softmax_cross_entropy(onehot_labels=tf.one_hot(y_batch,depth=10),logits=logits)

given a loss, create op that, when run, descends the gradient by one step

In [36]:
train_op = tf.train.GradientDescentOptimizer(learning_rate=.001).minimize(loss)

start a tensorflow session and use it to initialize all variables and ops

In [42]:
config = tf.ConfigProto(device_count={'GPU':0})
sess = tf.InterativeSession()
sess.run(tf.global_variables_initializer())

do gradient descent: run a train loop over multiple iterations

In [41]:
 sess.run(batch_generator.initializer,feed_dict{X_train_or_test: X_train, y_train_or_test: y_train})

In [ ]:
for i in range(1000):
    _, loss_i = sess.run([train_op,loss])
    if i%100 == 0:
        print(loss_i)

pass through the validation set

In [ ]:
sess.run(batch_generator.initializer, feed_dict{X_train_or_test:X_test, y_train_or_test:y_test})

In [45]:
correct = 0
total = 0

for i in range(100):
    prediction, truth = sess.run([logits,y_batch])
    correct += np.sum(prediction.argmax(1)==truth)
    total +=len(truth)

In [ ]:
print(correct/total)